In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gemini.json"

api_key = os.environ.get("OPENAI_API_KEY")
if api_key is None:
    raise RuntimeError("Please set the OPENAI_API_KEY ")

In [2]:
from scirag import SciRagHybrid, SciRagDataSet

In [3]:
import pandas as pd
import time
import datetime

In [4]:
def parse_response(response):
    """Parse response to extract answer and sources"""
    if not response:
        return None, None
    
    try:
        if "**Answer**:" in response and "**Sources**:" in response:
            parts = response.split("**Sources**:")
            answer = parts[0].replace("**Answer**:", "").strip()
            sources = parts[1].strip() if len(parts) > 1 else ""
            return answer, sources
        else:
            return response, ""
    except:
        return response, ""

Comparison of two type of embeddings, if we want to load the embeddings, and rerun, then we could download the embeddings from Google Drive

# OpenAI Embedding

In [7]:
def get_openAI_embedding_response(num_questions=105):
    """
    Returns separate DataFrames for OpenAI embeddings, answered by a Gimini RAG system.
    
    Returns:
        pd.DataFrame: DataFrame containing OpenAI RAG responses.
    """
    
    # Load dataset
    dataset = SciRagDataSet()
    df = dataset.load_dataset()
    
    # Get questions
    question_columns = ['question', 'query', 'text', 'prompt']
    question_col = None
    for col in question_columns:
        if col in df.columns:
            question_col = col
            break
    
    if question_col is None:
        text_cols = df.select_dtypes(include=['object']).columns
        question_col = text_cols[0] if len(text_cols) > 0 else df.columns[0]
    
    questions = df[question_col].head(num_questions).tolist()
    
    # Initialize RAG systems
    print("Initializing RAG systems...")
    openai_rag = SciRagHybrid(embedding_provider="openai", openai_embedding_model="text-embedding-3-large", vector_db_backend="chromadb")
    
    # Process questions
    openai_results = []
    
    for i, question in enumerate(questions, 1):
        print(f"Processing question {i}/{num_questions}")
        
        # OpenAI processing
        start_time = time.time()
        try:
            openai_response = openai_rag.get_response(question)
            openai_time = time.time() - start_time
            ideal_solution=df['ideal'].iloc[i-1] if 'ideal' in df.columns else None
            openai_answer, openai_sources = parse_response(openai_response)
            openai_success = True
            openai_error = None
        except Exception as e:
            openai_response = None
            openai_time = time.time() - start_time
            openai_answer = None
            openai_sources = None
            openai_success = False
            openai_error = str(e)
        
        openai_results.append({
            'question_id': i,
            'question': question,
            'response': openai_response,
            'ideal_solution': ideal_solution,
            'answer': openai_answer,
            'sources': openai_sources,
            'processing_time': openai_time,
            'success': openai_success,
            'error': openai_error,
            'embedding_system': 'OpenAI'
        })
    
    # Create DataFrames
    openai_df = pd.DataFrame(openai_results)
    return openai_df

In [9]:
#load the original OpenAI results
openai_df = pd.read_pickle("results/openai_embedding_results.pkl")

In [10]:
#check whether all questions were successfully answered
openai_df['success'].value_counts().to_dict()

openai_df[openai_df['success'] == False]['question_id'].tolist()
#check which questios 

[81, 85, 89, 95, 104]

In [11]:
dataset = SciRagDataSet()
qa = dataset.load_dataset()

In [12]:
qids=[80,84,88,94,103]

In [13]:
openai_rag = SciRagHybrid(embedding_provider="openai", openai_embedding_model="text-embedding-3-large", vector_db_backend="chromadb")

  Loaded 2503.14454v1.md
  Loaded 2010.00619v2.md
  Loaded 2201.02202v1.md
  Loaded 1807.06209v4.md
  Loaded 1604.01424v3.md
Total markdown documents loaded: 5
Splitting documents into chunks...
Created 305 chunks from 5 documents
Building contextual retrieval index...
Processed 305 chunks
Initializing embeddings and vector database...
Checking for existing embeddings at: /Users/apple/Documents/GitHub/scirag_paperqa/scirag/embeddings/openai_text-embedding-3-large_embeddings.npy
Loading existing embeddings from: /Users/apple/Documents/GitHub/scirag_paperqa/scirag/embeddings/openai_text-embedding-3-large_embeddings.npy
Creating vector database...
Loaded existing ChromaDB collection: sci_rag_chunks_openai_text_embedding_3_large
Loaded existing ChromaDB collection


In [14]:
openai_results = []
for qid in qids:
    print(f"Processing question {qid}")
        
    # OpenAI processing
    start_time = time.time()
    try:
        question = qa['question'].iloc[qid]  
        openai_response = openai_rag.get_response(question)
        openai_time = time.time() - start_time
        ideal_solution=qa['ideal'].iloc[qid] if 'ideal' in qa.columns else None
        openai_answer, openai_sources = parse_response(openai_response)
        openai_success = True
        openai_error = None
    except Exception as e:
        openai_response = None
        openai_time = time.time() - start_time
        openai_answer = None
        openai_sources = None
        openai_success = False
        openai_error = str(e)
    
    openai_results.append({
        'question_id': qid+1,  # Adjusting to match the original index
        'question': question,
        'response': openai_response,
        'ideal_solution': ideal_solution,
        'answer': openai_answer,
        'sources': openai_sources,
        'processing_time': openai_time,
        'success': openai_success,
        'error': openai_error,
        'embedding_system': 'OpenAI'
    })
# Create DataFrame for the new results
openai_df_additional = pd.DataFrame(openai_results)
    

Processing question 80
Processing question 84
Processing question 88
Processing question 94
Processing question 103


In [17]:
openai_df

,question_id,question,response,ideal_solution,answer,sources,processing_time,success,error,embedding_system
0,1,How is the standard recombination history test...,**Answer**:\n\nThe Planck 2018 analysis tests ...,semi-blind eigen-analysis (often referred to a...,The Planck 2018 analysis tests the standard re...,"1807.06209v4.md, Section 7.7, 1807.06209v4.md,...",6.386073,True,None,OpenAI
1,2,Which corrections in polarization spectra were...,"**Answer**:\n\nIn the 2018 Planck analysis, co...",Beam leakage correction; effective polarizatio...,"In the 2018 Planck analysis, corrections appli...",1807.06209v4.md,9.350544,True,None,OpenAI
2,3,What multipole cuts were applied in the Camspe...,"**Answer**:\n\nFor the Planck 2018 analysis, t...","lmin=500, lmax=2500","For the Planck 2018 analysis, the Camspec temp...",1807.06209v4.md,3.676692,True,None,OpenAI
3,4,What is the effective sky-fraction of the apod...,"**Answer**:\n\nFor the Planck 2018 analysis, t...",47.70%,"For the Planck 2018 analysis, the default CamS...",1807.06209v4.md,2.763290,True,None,OpenAI
4,5,How large is the impact of beam window functio...,"**Answer**:\n\nIn the 2018 release, the applic...",approximately 0.1% at l=2000,"In the 2018 release, the application of beam w...",1807.06209v4.md,3.579194,True,None,OpenAI
...,...,...,...,...,...,...,...,...,...,...
100,101,What would be the signs of dark matter annihil...,**Answer**:\n\nDark matter annihilation into S...,Damping of high-l power in both temperature an...,Dark matter annihilation into Standard Model p...,"2503.14454v1.md, Section 6.4.2, 1807.06209v4.m...",3.928702,True,None,OpenAI
101,102,"In the ACT DR6 paper, why is the constraint fo...",**Answer**:\n\nThe constraint on N<sub>idr</su...,Bayesian constraints on the IDR-IDM model caus...,The constraint on N<sub>idr</sub> for the IDR-...,"2503.14454v1.md, Section 6.5.2, Figure 35 capt...",6.198485,True,None,OpenAI
102,103,Why does including f\sigma_8 measurements into...,**Answer**:\n\nIncluding fσ8 measurements dram...,The f\sigma_8 dataset contains two low-z point...,Including fσ8 measurements dramatically increa...,"2503.14454v1.md, Figure 39, 2503.14454v1.md, S...",5.217752,True,None,OpenAI
103,104,Do the extensions to \lambdaCDM considered in ...,None,The f\sigma_8 dataset contains two low-z point...,None,None,267.641682,False,Unterminated string starting at: line 2 column...,OpenAI


In [16]:
openai_df_additional

,question_id,question,response,ideal_solution,answer,sources,processing_time,success,error,embedding_system
0,81,What is the final value of H_0 quoted in the H...,**Answer**:\n\nThe final value of H\n0 quoted ...,73.24 \pm 1.59 km s^{-1} Mpc^{-1},The final value of H\n0 quoted in the HST pape...,1604.01424v3.md,6.434770,True,None,OpenAI
1,85,What is the degree of tension between the loca...,**Answer**:\n\nThe HST paper reports a local H...,3.4\sigma,The HST paper reports a local Hubble Constant ...,1604.01424v3.md,6.751314,True,None,OpenAI
2,89,"In the ACT DR6 paper, what code libraries are ...",**Answer**:\n\nThe ACT DR6 paper utilizes mult...,"HyRec, CosmoRec and PRIMAT. HMcode is used in ...",The ACT DR6 paper utilizes multiple Einstein-B...,"2503.14454v1.md:Appendix A, 2503.14454v1.md:Fi...",6.146280,True,None,OpenAI
3,95,In the n=3 EDE model considered in the ACT DR6...,"**Answer**:\n\nIn the n=3 EDE model, the value...",67.5^{+0.9}_{-1.7},"In the n=3 EDE model, the value of H\n0 from t...",2503.14454v1.md,3.370523,True,None,OpenAI
4,104,Do the extensions to \lambdaCDM considered in ...,"**Answer**:\n\nNo, the extensions to the \(\La...",No. The range of H_0 given by the models studi...,"No, the extensions to the \(\Lambda\)CDM model...","2503.14454v1.md: Section 8.2, page 51, 2503.14...",5.973546,True,None,OpenAI


In [19]:
final_df = pd.concat([openai_df, openai_df_additional], ignore_index=True)

In [30]:
#drop the one with success == False
final_df = final_df[final_df['success'] == True]
final_df.sort_values(by='question_id', inplace=True)
final_df.reset_index(drop=True, inplace=True)
final_df['question_id'].value_counts().to_dict()

# Save the final DataFrame
final_df.to_pickle("results/openai_embedding_results_final.pkl")

# Gemini embedding


In [ ]:
def get_Gemini_embedding_response(num_questions=105):
    """
    Returns DataFrames for Gemini embedding, gemini RAG system. 
    
    Returns:
        pd.DataFrame: DataFrame containing Gemini RAG responses.
    """
    
    # Load dataset
    dataset = SciRagDataSet()
    df = dataset.load_dataset()
    
    # Get questions
    question_columns = ['question', 'query', 'text', 'prompt']
    question_col = None
    for col in question_columns:
        if col in df.columns:
            question_col = col
            break
    
    if question_col is None:
        text_cols = df.select_dtypes(include=['object']).columns
        question_col = text_cols[0] if len(text_cols) > 0 else df.columns[0]
    
    questions = df[question_col].head(num_questions).tolist()
    
    # Initialize RAG systems
    print("Initializing RAG systems...")
    gemini_rag = SciRagHybrid(embedding_provider="gemini", vector_db_backend="chromadb") #we didn't specify the embedding, so it will use the default one in the config file, please check the config file for the embedding model used
    
    # Process questions
    gemini_results = []

    
    for i, question in enumerate(questions, 1):
        print(f"Processing question {i}/{num_questions}")
        
        # Gemini processing
        start_time = time.time()
        try:
            gemini_response = gemini_rag.get_response(question)
            gemini_time = time.time() - start_time
            gemini_answer, gemini_sources = parse_response(gemini_response)
            ideal_solution=df['ideal'].iloc[i-1] if 'ideal' in df.columns else None
            gemini_success = True
            gemini_error = None
        except Exception as e:
            gemini_response = None
            gemini_time = time.time() - start_time
            gemini_answer = None
            gemini_sources = None
            gemini_success = False
            gemini_error = str(e)
        
        gemini_results.append({
            'question_id': i,
            'question': question,
            'ideal_solution': ideal_solution,
            'response': gemini_response,
            'answer': gemini_answer,
            'sources': gemini_sources,
            'processing_time': gemini_time,
            'success': gemini_success,
            'error': gemini_error,
            'embedding_system': 'Gemini'
        })
    
    # Create DataFrames
    gemini_df = pd.DataFrame(gemini_results)
    return gemini_df

In [39]:
gemini_df= get_Gemini_embedding_response(num_questions=105)
# Save the Gemini results
gemini_df.to_pickle("results/gemini_embedding_results.pkl")

Initializing RAG systems...
  Loaded 2503.14454v1.md
  Loaded 2010.00619v2.md
  Loaded 2201.02202v1.md
  Loaded 1807.06209v4.md
  Loaded 1604.01424v3.md
Total markdown documents loaded: 5
Splitting documents into chunks...
Created 305 chunks from 5 documents
Building contextual retrieval index...
Processed 305 chunks
Initializing embeddings and vector database...
Checking for existing embeddings at: /Users/apple/Documents/GitHub/scirag_paperqa/scirag/embeddings/gemini_gemini-embedding-001_embeddings.npy
Loading existing embeddings from: /Users/apple/Documents/GitHub/scirag_paperqa/scirag/embeddings/gemini_gemini-embedding-001_embeddings.npy
Creating vector database...
Loaded existing ChromaDB collection: sci_rag_chunks_gemini_gemini_embedding_001
Loaded existing ChromaDB collection
Processing question 1/105
Processing question 2/105
Processing question 3/105
Processing question 4/105
Processing question 5/105
Processing question 6/105
Processing question 7/105
Processing question 8/105

In [41]:
#check the success of the Gemini results
gemini_df[gemini_df['success'] == False]['question_id'].tolist()

[]

# OpenAI-Vector-Store

openAI-rag

In [10]:
from scirag import SciRagOpenAI

In [11]:
def get_OpenAI_rag_response(num_questions=105):
    """
    uses the default OpenAI Vector store to answer questions using the SciRagOpenAI class.
    Returns:
        pd.DataFrame: DataFrame containing OpenAI RAG responses.
    """
    
    # Load dataset
    dataset = SciRagDataSet()
    df = dataset.load_dataset()
    
    # Get questions
    question_columns = ['question', 'query', 'text', 'prompt']
    question_col = None
    for col in question_columns:
        if col in df.columns:
            question_col = col
            break
    
    if question_col is None:
        text_cols = df.select_dtypes(include=['object']).columns
        question_col = text_cols[0] if len(text_cols) > 0 else df.columns[0]
    
    questions = df[question_col].head(num_questions).tolist()
    
    # Initialize RAG system
    print("Initializing OpenAI RAG system...")
    scirag = SciRagOpenAI(vector_db_backend="openai")
    
    openai_results = []
    
    for i, question in enumerate(questions, 1):
        print(f"Processing question {i}/{num_questions}")
        ideal_solution=df['ideal'].iloc[i-1] if 'ideal' in df.columns else None
        
        # OpenAI processing
        start_time = time.time()
        try:
            openai_response = scirag.get_response(question)
            openai_time = time.time() - start_time
            openai_answer, openai_sources = parse_response(openai_response)

            openai_success = True
            openai_error = None
        except Exception as e:
            openai_response = None
            openai_time = time.time() - start_time
            
            openai_answer = None
            openai_sources = None
            openai_success = False
            openai_error = str(e)
        
        openai_results.append({
            'question_id': i,
            'question': question,
            'ideal_solution': ideal_solution,
            'response': openai_response,
            'answer': openai_answer,
            'sources': openai_sources,
            'processing_time': openai_time,
            'success': openai_success,
            'error': openai_error,
            'embedding_system': 'OpenAI'
        })
    
    # Create DataFrame
    openai_df = pd.DataFrame(openai_results)
    return openai_df

In [12]:
openai_df=get_OpenAI_rag_response(num_questions=105)
# Save the OpenAI RAG results
openai_df.to_pickle("results/openai_pdf_vector_store_results.pkl")

Initializing OpenAI RAG system...
Listing existing RAG Corpora:
--- Found existing corpus: corpus ---
Creating assistant for existing vector store...
Assistant created for existing corpus with ID: asst_AiTRTGJCNZyM7380vYaKbPfd
Processing question 1/105
Processing question 2/105
Processing question 3/105
Processing question 4/105
Processing question 5/105
Processing question 6/105
Processing question 7/105
Processing question 8/105
Processing question 9/105
Processing question 10/105
Processing question 11/105
Processing question 12/105
Processing question 13/105
Processing question 14/105
Processing question 15/105
Processing question 16/105
Processing question 17/105
Processing question 18/105
Processing question 19/105
Processing question 20/105
Processing question 21/105
Processing question 22/105
Processing question 23/105
Processing question 24/105
Processing question 25/105
Processing question 26/105
Processing question 27/105
Processing question 28/105
Processing question 29/105

In [13]:
openai_df[openai_df['success'] == False]['question_id'].tolist()

[]

# Paperqa2 response

In [5]:
from scirag import SciRagPaperQA2

In [6]:
from IPython.display import display, Markdown

In [7]:
def get_PaperQA2_rag_response(num_questions=105):
# Load dataset
    dataset = SciRagDataSet()
    df = dataset.load_dataset()
    
    # Get questions - using 'question' column based on your example
    question_columns = ['question', 'query', 'text', 'prompt']
    question_col = None
    for col in question_columns:
        if col in df.columns:
            question_col = col
            break
    
    if question_col is None:
        text_cols = df.select_dtypes(include=['object']).columns
        question_col = text_cols[0] if len(text_cols) > 0 else df.columns[0]
    
    questions = df[question_col].head(num_questions).tolist()
    
    # Initialize PaperQA2 RAG system
    print("Initializing PaperQA2 RAG system...")
    scirag = SciRagPaperQA2()
    
    paperqa2_results = []
    
    for i, question in enumerate(questions, 1):
        print(f"Processing question {i}/{num_questions}")
        print(f"Question: {question}")
        ideal_solution=df['ideal'].iloc[i-1] if 'ideal' in df.columns else None
        
        # PaperQA2 processing with timing
        start_time = time.time()
        try:
            paperqa2_response = scirag.get_response(question)
            paperqa2_time = time.time() - start_time
            paperqa2_answer, paperqa2_sources = parse_response(paperqa2_response.answer)
            paperqa2_success = True
            paperqa2_error = None
            
            # Display results for interactive use
            display(Markdown(f"**Response:**\n{paperqa2_response.answer}"))
            
        except Exception as e:
            paperqa2_response = None
            paperqa2_time = time.time() - start_time
            paperqa2_answer = None
            paperqa2_sources = None
            paperqa2_success = False
            paperqa2_error = str(e)
        print(f"Processing Time: {paperqa2_time:.2f} seconds")
        
        paperqa2_results.append({
            'question_id': i,
            'question': question,
            'response': paperqa2_response.answer,
            'answer': paperqa2_answer,
            'sources': paperqa2_sources,
            'ideal_solution': ideal_solution,
            'processing_time': paperqa2_time,
            'success': paperqa2_success,
            'error': paperqa2_error,
            'embedding_system': 'PaperQA2'
        })
    
    
    # Create DataFrame
    paperqa2_df = pd.DataFrame(paperqa2_results)
    
    return paperqa2_df

In [8]:
paperqa2_df = get_PaperQA2_rag_response(num_questions=105)

Initializing PaperQA2 RAG system...
[SciRagPaperQA2] Building index on initialization...
[SciRagPaperQA2] Checking for paper directory: /Users/apple/Documents/GitHub/scirag_paperqa/scirag/txt_files
[SciRagPaperQA2] Building PaperQA2 document index (only happens once)...
Using index: pqa_index_969208873c16842f6232653a9eca7e7a
Index files: {'2201.02202v1.txt': 'a569bb587a9091300ba2f479aba3efc8', '1604.01424v3.txt': '70196f1a55258dc0d6ece83887dec601', '1807.06209v4.txt': '61f1a51953008a179d876dded8a7f3a1', '2010.00619v2.txt': 'cc0b6c6c76fe9a1b9132c0dd9de161bf', '2503.14454v1.txt': 'cc44e168db0f207acfe5c6a6f185e266'}
[SciRagPaperQA2] Index built successfully.
Processing question 1/105
Question: How is the standard recombination history tested in the Planck 2018 analysis?


[16:37:09] Starting paper search for 'Planck 2018 recombination history test'.

           paper_search for query 'Planck 2018 recombination history test' and offset 0 returned 5 papers.

           Status: Paper Count=5 | Relevant Papers=0 | Current Evidence=0 | Current Cost=$0.0015

[16:37:10] gather_evidence starting for question 'How is the standard recombination history tested in the Planck   
           2018 analysis?'.

...

[18:07:44] Status: Paper Count=5 | Relevant Papers=2 | Current Evidence=3 | Current Cost=$0.0851

[18:07:45] Completing 'What is the CMB monopole temperature at the 68% CL, according to the P-ACT-LB dataset?' as  
           'certain'.

           Answer: The CMB monopole temperature at the 68% confidence level (CL), according to the P-ACT-LB        
           dataset, is \( T_{\mathrm{CMB}} = 2.698 \pm 0.016 \) K (calabrese2025theatacamacosmology chunk 2).

**Response:**
**Answer**:

The CMB monopole temperature at the 68% confidence level (CL), according to the P-ACT-LB dataset, is \( T_{\mathrm{CMB}} = 2.698 \pm 0.016 \) K .

**Sources**:

calabrese2025theatacamacosmology chunk 2


Processing Time: 46.14 seconds


In [9]:
paperqa2_df.to_pickle("results/paperqa2_gpt4.1_results.pkl")

In [11]:
paperqa2_df

,question_id,question,response,answer,sources,ideal_solution,processing_time,success,error,embedding_system
0,1,How is the standard recombination history test...,**Answer**:\n\nThe Planck 2018 analysis tests ...,The Planck 2018 analysis tests the standard re...,aghanim2020planck2018results chunk 63; aghanim...,semi-blind eigen-analysis (often referred to a...,20.449642,True,None,PaperQA2
1,2,Which corrections in polarization spectra were...,"**Answer**:\n\nIn the 2018 Planck analysis, co...","In the 2018 Planck analysis, corrections to th...",aghanim2020planck2018results chunk 8; aghanim2...,Beam leakage correction; effective polarizatio...,59.101006,True,None,PaperQA2
2,3,What multipole cuts were applied in the Camspe...,"**Answer**:\n\nIn the Planck 2018 analysis, th...","In the Planck 2018 analysis, the CamSpec tempe...",aghanim2020planck2018results chunk 10; aghanim...,"lmin=500, lmax=2500",57.254722,True,None,PaperQA2
3,4,What is the effective sky-fraction of the apod...,**Answer**:\n\nThe effective sky-fraction of t...,The effective sky-fraction of the apodized Cam...,aghanim2020planck2018results chunk 11,47.70%,60.132730,True,None,PaperQA2
4,5,How large is the impact of beam window functio...,**Answer**:\n\nThe impact of beam window funct...,The impact of beam window functions on the 201...,aghanim2020planck2018results chunk 8; aghanim2...,approximately 0.1% at l=2000,57.501149,True,None,PaperQA2
...,...,...,...,...,...,...,...,...,...,...
100,101,What would be the signs of dark matter annihil...,**Answer**:\n\nSigns of dark matter annihilati...,Signs of dark matter annihilation into Standar...,aghanim2020planck2018results chunk 68; calabre...,Damping of high-l power in both temperature an...,45.888946,True,None,PaperQA2
101,102,"In the ACT DR6 paper, why is the constraint fo...","**Answer**:\n\nIn the ACT DR6 paper, the const...","In the ACT DR6 paper, the constraint on the ef...",calabrese2025theatacamacosmology chunk 39; cal...,Bayesian constraints on the IDR-IDM model caus...,98.421903,True,None,PaperQA2
102,103,Why does including f\sigma_8 measurements into...,**Answer**:\n\nIncluding fσ₈ measurements—prob...,Including fσ₈ measurements—probes of the growt...,calabrese2025theatacamacosmology chunk 43; agh...,The f\sigma_8 dataset contains two low-z point...,58.504149,True,None,PaperQA2
103,104,Do the extensions to \lambdaCDM considered in ...,**Answer**:\n\nThe extensions to ΛCDM consider...,The extensions to ΛCDM considered in the ACT D...,"calabrese2025theatacamacosmology chunk 4, cala...",No. The range of H_0 given by the models studi...,50.710328,True,None,PaperQA2


In [10]:
#check the success of the paperqa2 results
paperqa2_df[paperqa2_df['success'] == False]['question_id'].tolist()

[]